In [1]:
%matplotlib inline

import os
import numpy as np
import rasterio as rio
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [3]:
# Load the AOI polygon
AOI_path = 'Data_files/Area_of_interest.shp'
AOI = gpd.read_file(AOI_path)

#load the join shapefile
join_path = 'Data_files/joined_with_area.shp'
join = gpd.read_file(join_path)

# Load the clipped NDVI raster
ndvi_path = 'Data_files/clipped_ndvi.tif'

In [4]:
# Calculate the centroid of the area of interest
centroid = AOI.geometry.centroid
center_lat, center_lon = centroid.y.iloc[0], centroid.x.iloc[0]
print(f"Latitude: {center_lat}, Longitude: {center_lon}")

Latitude: 4528774.390699869, Longitude: 628697.575125249


In [5]:
import pyproj

# Define the UTM zone (e.g., UTM Zone 33N for Europe)
utm_zone = 33

# Create a UTM to latitude-longitude (WGS84) transformer
utm_to_wgs84 = pyproj.Transformer.from_crs(f'EPSG:326{utm_zone}', 'EPSG:4326', always_xy=True)

# Your provided UTM coordinates
utm_x, utm_y = center_lon, center_lat

# Convert to latitude and longitude
lon, lat = utm_to_wgs84.transform(utm_x, utm_y)

print(f"Latitude: {lat}, Longitude: {lon}")

Latitude: 40.89995038970218, Longitude: 16.52793000194325


In [15]:
with rio.open(ndvi_path) as ndvi_src:
    ndvi_data = ndvi_src.read(1)

# Create a Folium map centered at a specific location (e.g., latitude, longitude)
folium_map = folium.Map(location=[lat, lon], zoom_start=11)

In [16]:
# Transform the CRS to WGS84 (EPSG:4326)
AOI_wgs84 = AOI.to_crs('+proj=longlat +datum=WGS84')

In [ ]:
import matplotlib.colors as mpl_colors  # Import the mpl_colors module

AOI_geojson = AOI_wgs84.__geo_interface__

# Define custom colormap (adjust as needed)
cmap = plt.get_cmap('RdYlGn')  # Red-Yellow-Green colormap
norm = plt.Normalize(vmin=0, vmax=1)  # Normalize NDVI values to [0, 1]

# Compute colors for each NDVI value
ndvi_colors = [mpl_colors.rgb2hex(cmap(norm(val))) for val in ndvi_data.flatten()]

# Add GeoJSON layer with precomputed colors
folium.Choropleth(
    geo_data=AOI_geojson,
    data=ndvi_data.flatten(),
    key_on='feature.id',  # Adjust based on your GeoJSON structure
    fill_color='RdYlGn',  # Use precomputed colors
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='NDVI',
).add_to(folium_map)

# Display the map
folium_map

In [ ]:
# Convert the GeoDataFrame to a JSON-serializable format
geojson_data = join.to_crs(epsg='4326').to_json()

# Add the GeoJson layer to the map
folium.GeoJson(
    geojson_data,
    name='Agricultural Land and Protected Areas'
).add_to(folium_map)

# Display the map
folium_map

In [ ]:
# Save the map to an HTML file
folium_map.save('Data_files/ndvi_with_shapefile_map.html')

In [ ]:
join.explore('DESC_', popup=True, legend=False, cmap='viridis')